In [307]:
from glob import glob
import random
import uuid
import csv  
import os


from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [308]:
def generate_card(card_path, logo_path):
    back = Image.open(card_path).convert('RGBA')
    logo = Image.open(logo_path).convert('RGBA')

    logo_w = back.width / 4.7
    loho_h = logo_w / logo.width  * logo.height
    logo_w = np.round(logo_w).astype(int)
    loho_h = np.round(loho_h).astype(int)

    logo = logo.resize((logo_w, loho_h))

    xmin = np.random.randint(0, back.width - logo.width)
    xmax = xmin + logo.width
    ymin = np.random.randint(0, back.height - logo.height)
    ymax = ymin + logo.height

    back.paste(logo, (xmin, back.height - ymax), logo)
    return back, (xmin, xmax, ymin, ymax)

In [309]:
RESULT_CSV = './synth/generated.csv'

if os.path.isfile(RESULT_CSV):
    os.remove(RESULT_CSV)
    
if not os.path.exists('./synth/images'):
    os.makedirs('./synth/images')
    
files = glob('./synth/images/*')
for f in files:
    os.remove(f)
    
switcher = 0

for _ in range(1024):

    card_type = 'visa' if switcher else 'mastercard'

    card_pattern = './synth/card/*'
    logo_pattern = f'./synth/logo-{card_type}/*'
    switcher = 1 - switcher


    logo_path = random.choice(glob(logo_pattern))
    card_path = random.choice(glob(card_pattern))


    card, (xmin, xmax, ymin, ymax) = generate_card(card_path, logo_path)

    label_name = card_type
    rect_left = xmin
    rect_top = card.height - ymax
    rect_width = xmax - xmin
    rect_height = ymax - ymin
    image_name = uuid.uuid4().hex + '.png'
    image_width = card.width
    image_height = card.height
    
    card.save(f'./synth/images/{image_name}')
    
    row = [label_name, rect_left, rect_top, rect_width, rect_height, image_name, image_width, image_height]
    
    with open(RESULT_CSV, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(row)